# Guía básica para las llamadas de función en OpenAI

### 1. Configuración inicial

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import json

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Configurar la clave de la API de OpenAI
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)


### 2. Definir descripciones de funciones
Define las descripciones de las funciones que planeas utilizar en tu diálogo. Estas descripciones incluyen el nombre de la función, una breve descripción y detalles sobre los parámetros que espera la función.

In [2]:
function_descriptions = [
    {
        "name": "consultar_api_vuelos",
        "description": "Descripción de la función consultar_api_vuelos",
        "parameters": {
            # Detalles de los parámetros esperados
        },
    },
    # Otras descripciones de funciones...
]

### 3. Definir las funciones (ejemplo de consulta a una api)

In [3]:
import requests

def consultar_api_vuelos(url, params=None):
    """
    Realiza una consulta a una API utilizando la URL proporcionada y los parámetros (si los hay).

    Args:
    - url (str): La URL de la API a la que se va a realizar la consulta.
    - params (dict, opcional): Los parámetros de la consulta (por ejemplo, filtros o datos de búsqueda).

    Returns:
    - dict: Un diccionario que contiene la respuesta de la API, convertida a JSON.
    - None: Si ocurrió algún error durante la consulta.
    """
    try:
        # Realizar la solicitud GET a la API con los parámetros proporcionados
        response = requests.get(url, params=params)

        # Verificar si la solicitud fue exitosa (código de estado 200)
        if response.status_code == 200:
            # Convertir la respuesta a JSON y devolverla
            return response.json()
        else:
            # Imprimir un mensaje de error si la solicitud no fue exitosa
            print(f"Error al consultar la API. Código de estado: {response.status_code}")
            return None
    except Exception as e:
        # Capturar cualquier excepción que ocurra durante la consulta
        print(f"Error al realizar la consulta a la API: {e}")
        return None
    
    # OTRAS FUNCIONES....

### 4. Interactuar con el modelo de lenguaje

In [4]:
user_prompt = "When's the next flight from Amsterdam to New York?"

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": user_prompt}],
    functions=function_descriptions,
    function_call="auto",  # especificar la llamada de función automática
)
output = completion.choices[0].message


### 5. Llamadas de función manual

In [5]:
params = json.loads(output.function_call.arguments)

chosen_function = eval(output.function_call.name)
flight = chosen_function(**params)

TypeError: consultar_api_vuelos() missing 1 required positional argument: 'url'

### 6. Incorporar resultados de función en la respuesta

In [ ]:
second_completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "function", "name": output.function_call.name, "content": flight},
    ],
    functions=function_descriptions,
)
response = second_completion.choices[0].message.content


### 7. Utilizar la función en un flujo de conversación (estas funciones sirven para realizar lo anterior)
Con la primera función obtendremos la función que se va a utilizar y los parámetros recogidos del user_prompt. Con la segunda función obtendremos la salida de la función utilizado y devolveremos el contenido al usuario en formato frase.

In [ ]:
def detect_function(prompt):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": prompt}],
        functions=function_descriptions,
        function_call="auto",  # especificar la llamada de función automática
    )
    output = completion.choices[0].message
    return output

def function_calling(prompt, function, content):
    """Give LLM a given prompt and get an answer."""

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": prompt},
                  {"role":"function", "name": function, "content": content}],

        functions=function_descriptions,
        function_call="auto",  # specify the function call
    )

    output = completion.choices[0].message
    return output


# EJEMPLO:

### 1. Cargamos las librerías y nos conectamos a la api de OpenAI

In [ ]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta

# Load environment variables from the .env file
load_dotenv()

# Configure the OpenAI API key
client = OpenAI(
    api_key=os.getenv
    ("OPENAI_API_KEY"),
)

GPT_MODEL = "gpt-3.5-turbo-1106"

### 2. Describimos las funciones que vamos a utilizar

In [ ]:

function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    },
    {
        "name": "book_flight",
        "description": "Book a flight based on flight information",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
                "datetime": {
                    "type": "string",
                    "description": "The date and time of the flight, e.g. 2023-01-01 01:01",
                },
                "airline": {
                    "type": "string",
                    "description": "The service airline, e.g. Lufthansa",
                },
            },
            "required": ["loc_origin", "loc_destination", "datetime", "airline"],
        },
    },
    {
        "name": "file_complaint",
        "description": "File a complaint as a customer",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the user, e.g. John Doe",
                },
                "email": {
                    "type": "string",
                    "description": "The email address of the user, e.g. john@doe.com",
                },
                "text": {
                    "type": "string",
                    "description": "Description of issue",
                },
            },
            "required": ["name", "email", "text"],
        },
    },
]


### 3. Definimos las funciones (en este caso se han creado ejemplos de respuesta de consulta de una api)

In [ ]:
def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "KLM",
        "flight": "KL643",
    }

    return json.dumps(flight_info)

def book_flight(loc_origin, loc_destination, datetime, airline):
    """Book a flight based on flight information."""

    # Example output returned from an API or database
    booking_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": datetime,
        "airline": airline,
        "flight": "KL643",
        "booking_reference": "ABC123",
    }

    return json.dumps(booking_info)

def file_complaint(name, email, text):
    """File a complaint as a customer."""

    # Example output returned from an API or database
    complaint_info = {
        "name": name,
        "email": email,
        "text": text,
        "complaint_id": "XYZ789",
    }

    return json.dumps(complaint_info)

### 4. Definimos las funciones que vamos a usar para detectar la función que requiere el user_prompt y la respuesta generada.

In [ ]:
def detect_function(prompt):
    """Give LLM a given prompt and get an answer."""

    completion = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[{"role": "user", "content": prompt}],

        functions=function_descriptions,
        function_call="auto",  # specify the function call
    )

    output = completion.choices[0].message
    return output

def function_calling(prompt, function, content):
    """Give LLM a given prompt and get an answer."""

    completion = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[{"role": "user", "content": prompt},
                  {"role":"function", "name": function, "content": content}],

        functions=function_descriptions,
        function_call="auto",  # specify the function call
    )

    output = completion.choices[0].message
    return output


### Creamos ejemplos de user_prompt se lo pasamos a la función que detecta que función vamos a utilizar para adquirir la información y luego realizamos la llamada a la misma

In [ ]:
# Scenario 1: File a complaint

user_prompt = "This is John Doe. I want to file a complaint about my missed flight. It was an unpleasant surprise. Email me a copy of the complaint to john@doe.com."
output = detect_function(user_prompt)
chosen_function = eval(output.function_call.name)

params = json.loads(output.function_call.arguments)
complaint = chosen_function(**params)

output = function_calling(user_prompt, output.function_call.name, complaint)
print(output.content)


Your complaint has been filed, and a copy has been emailed to you. You should receive an email shortly with the details.


In [ ]:
# Scenario 2: Check flight details

user_prompt = "When's the next flight from Amsterdam to New York?"
output = detect_function(user_prompt)


# Get info for the next prompt
origin = json.loads(output.function_call.arguments).get("loc_origin")
destination = json.loads(output.function_call.arguments).get("loc_destination")
flight_datetime = json.loads(flight).get("datetime")
flight_airline = json.loads(flight).get("airline")



chosen_function = eval(output.function_call.name)

params = json.loads(output.function_call.arguments)
flight = chosen_function(**params)

output = function_calling(user_prompt, output.function_call.name, flight)
print(output.content)



The next flight from Amsterdam (AMS) to New York (JFK) is on February 23, 2024, at 16:20 with KLM (Flight KL643).


In [ ]:
# Scenario 3: Book a new flight

user_prompt = f"I want to book a flight from {origin} to {destination} on {flight_datetime} with {flight_airline}"
output = detect_function(user_prompt)

chosen_function = eval(output.function_call.name)

params = json.loads(output.function_call.arguments)
booking = chosen_function(**params)
print(booking)

output = function_calling(user_prompt, output.function_call.name, booking)
print(output.content)

{"loc_origin": "AMS", "loc_destination": "JFK", "datetime": "2024-02-23 14:28:51.046294", "airline": "KLM", "flight": "KL643", "booking_reference": "ABC123"}
Your flight from AMS to JFK on 2024-02-23 at 14:28:51 with KLM has been successfully booked. Your booking reference is ABC123, and your flight number is KL643.


## Bibliografía

Vídeo interesante: "OpenAI Function Calling - Full Beginner Tutorial" https://www.youtube.com/watch?v=aqdWSYWC_LI


Página de referencia: https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models